In [8]:
%load_ext dotenv 
%dotenv
import os
import warnings
import openai
import textwrap
import pandas as pd

warnings.filterwarnings('ignore')
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
# langchain imports
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.indexes import VectorstoreIndexCreator

In [55]:
df = pd.read_csv("project_descriptions.csv")
df["length"] = df["content"].apply(lambda x: len(x.split(" ")))
df = df.drop(columns=["qa_list"])
df = df.head(20)
df.to_csv("project_descriptions_clean.csv", index=False)

In [56]:
loader = CSVLoader(file_path="project_descriptions_clean.csv", source_column="content")

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [57]:
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=index.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### Generate sample question aswer pairs for provided documents

In [60]:
from langchain.evaluation.qa import QAGenerateChain

example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

data = loader.load()
# data

new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t.page_content} for t in data[:10]]
)

In [61]:
new_examples

[{'query': 'What is the current market cap of Bitcoin and how many BTC coins are in circulation?',
  'answer': 'The current live market cap of Bitcoin is $330,270,059,122 USD and there are 19,230,868 BTC coins in circulation with a max. supply of 21,000,000 BTC coins.'},
 {'query': 'What is Ethereum and how does it differ from other cryptocurrencies?',
  'answer': 'Ethereum is a decentralized open-source blockchain system that features its own cryptocurrency, Ether, and works as a platform for numerous other cryptocurrencies, as well as for the execution of decentralized smart contracts. Ethereum differs from other'},
 {'query': 'What is Tether and how does it differ from other cryptocurrencies?',
  'answer': 'Tether (USDT) is a stablecoin that mirrors the price of the U.S. dollar and is issued by a Hong Kong-based company, Tether. Its unique feature is its value being guaranteed by Tether to remain pegged to the U.S. dollar. This is achieved by maintaining a sum of commercial paper, f

In [62]:
query = "What are some of the best crypto projects related to artificial intelligence and why?"

response = index.query(query)
display(Markdown(response))

 I don't know.

#### Simple chain for chat

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0,
    model='gpt-3.5-turbo'
)

In [35]:
query = "What are some of the best crypto projects related to artificial intelligence and why? give me a list of at least 5"

messages = [
    SystemMessage(content="You are a helpful assistant who is an expert in cryptocurrencies."),
    HumanMessage(content=query)
]
response = chat(messages)

In [34]:
display(Markdown(response.content))

Sure, here are 10 crypto projects related to artificial intelligence:

1. SingularityNET (AGI) - This project aims to create a decentralized marketplace for AI services, allowing developers to monetize their AI algorithms and models.

2. Fetch.ai (FET) - This project is building a decentralized network for autonomous economic agents, which can perform tasks such as data analysis and prediction using AI.

3. Ocean Protocol (OCEAN) - This project is creating a decentralized data exchange protocol that uses AI to help users discover and monetize their data.

4. Numerai (NMR) - This project is a hedge fund that uses AI to make investment decisions. It also allows data scientists to contribute their own AI models to the fund.

5. Cortex (CTXC) - This project is building a decentralized AI platform that allows developers to create and deploy AI models on the blockchain.

6. DeepBrain Chain (DBC) - This project is building a decentralized AI computing platform that allows users to rent out their computing power to train AI models.

7. Matrix AI Network (MAN) - This project is building a blockchain-based AI platform that allows developers to create and deploy AI models using natural language programming.

8. Neuromation (NTK) - This project is building a decentralized platform for AI model training and development, using blockchain technology to ensure data privacy and security.

9. Endor Protocol (EDR) - This project is building a decentralized predictive analytics platform that uses AI to help businesses make better decisions.

10. AICoin (XAI) - This project is a cryptocurrency investment fund that uses AI to make investment decisions, with profits reinvested into the development of AI technology.

These projects are all focused on using blockchain technology to improve the development and deployment of AI, and have the potential to revolutionize industries such as finance, healthcare, and more.